In [ ]:
%matplotlib inline

In [ ]:
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 

In [ ]:
train_df = pd.read_csv("train.csv")

In [ ]:
len(train_df)

In [ ]:
tokenizer = Tokenizer()

In [ ]:
train = train_df[:100]

In [ ]:
tokenizer.fit_on_texts(list(train["comment_text"]))

word_2_index = tokenizer.word_index
index_2_word = {ix: word for word, ix in word_2_index.items()}

In [ ]:
tokens = tokenizer.texts_to_sequences(train["comment_text"])

Load fast text weights from downloaded file

In [ ]:
fasttext_weights = pd.read_csv("fasttext/fasttext.vec", sep=" ", skiprows=1,
                               quoting=csv.QUOTE_NONE, header=None, index_col=0)

In [ ]:
# Reading CSV puts an empty column at the end. Drop that column.
fasttext_weights.drop(labels=len(fasttext_weights.columns), axis=1, inplace=True)

In [ ]:
embedding_weights = np.zeros((len(index_2_word) + 1, 300))

for word, index in word_2_index.items():
    if word in fasttext_weights.index:
        embedding_weights[index] = fasttext_weights.loc[word]


In [162]:
train_X = pad_sequences(tokens, 75)

In [163]:
train_y = [[train.iloc[row]["toxic"], train.iloc[row]["severe_toxic"],
                     train.iloc[row]["obscene"], train.iloc[row]["threat"], 
                     train.iloc[row]["insult"], train.iloc[row]["identity_hate"]
                     ]for row in range(len(train))]

In [164]:
model = Sequential()

embedding = Embedding(len(word_2_index) + 1, 300, input_length=75)

model.add(embedding)
# model.add(Dense(32, input_shape=(10,)))
model.add(Dense(6, activation="sigmoid"))

In [165]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 75, 300)           504600    
_________________________________________________________________
dense_31 (Dense)             (None, 75, 6)             1806      
Total params: 506,406
Trainable params: 506,406
Non-trainable params: 0
_________________________________________________________________


In [166]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [167]:
model.fit(x=np.array(train_X), y=np.array(train_y))

ValueError: Error when checking target: expected dense_31 to have 3 dimensions, but got array with shape (100, 6)

In [132]:
?Dense